In [1]:
import numpy as np
import xgboost as xgb
import pandas as pd
import sklearn
from sklearn.preprocessing import LabelEncoder
from sklearn import metrics
from sklearn.model_selection import GridSearchCV
from sklearn.utils.multiclass import unique_labels
import category_encoders as ce

/Users/jarroyo/miniforge3/lib/python3.9/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index
/Users/jarroyo/miniforge3/lib/python3.9/site-packages/statsmodels/compat/pandas.py:65: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import Int64Index as NumericIndex


Convert data to numeric:
- ordinal variables encoded as

In [2]:
train_data = pd.read_csv('../data/LOANS_TRAIN.csv')
test_data = pd.read_csv('../data/LOANS_TEST.csv')

id_column = test_data['id']

train_data.drop(columns=['id','grade', 'emp_title', 'title', 'earliest_cr_line', 'issue_d'], axis=1, inplace=True)
test_data.drop(columns=['id','grade', 'emp_title', 'title', 'earliest_cr_line', 'issue_d'], axis=1, inplace=True)

# we want to ultimately use this data, but its nominal multi-categorical nature requires further preprocessing
# prolly with OneHotEncoder
    # Deal with nominal, multi-categorical data
    # Goal: convert home_ownership (RENT, OWN, MORTGAGE, OTHER) to a 4D vector.
        # if MORTGAGE ==> [0,0,1,0]
# print(len(train_data['zip_code'].unique()))
# print(train_data['zip_code'].unique())
ce_OHE = ce.OneHotEncoder(cols=['addr_state', 'home_ownership', 'zip_code'])
print('BEFORE')
print(train_data.shape)
print(train_data.iloc[0])
train_data = ce_OHE.fit_transform(train_data)
print('AFTER')
print(train_data.shape)
print(train_data.keys())
test_data = ce_OHE.fit_transform(test_data)
train_data.drop(columns=['purpose'], axis=1, inplace=True)
test_data.drop(columns=['purpose'], axis=1, inplace=True)
# NOTE: 'purpose' only has 14 categories, probably good things to learn from

# there are no joint applications in training, so model won't be able to learn it. Better to drop it
train_data.drop(columns=['application_type'], axis=1, inplace=True)
test_data.drop(columns=['application_type'], axis=1, inplace=True)

labelencoder = LabelEncoder()
labelencoder2 = LabelEncoder()
# Assigning numerical values and storing in another column
train_data['sub_grade'] = labelencoder.fit_transform(train_data['sub_grade'])
train_data['emp_length'].replace('< 1 year', 0, inplace=True)
train_data['emp_length'].replace('1 year', 1.0, inplace=True)
train_data['emp_length'].replace('2 years', 2.0, inplace=True)
train_data['emp_length'].replace('3 years', 3.0, inplace=True)
train_data['emp_length'].replace('4 years', 4.0, inplace=True)
train_data['emp_length'].replace('5 years', 5.0, inplace=True)
train_data['emp_length'].replace('6 years', 6.0, inplace=True)
train_data['emp_length'].replace('7 years', 7.0, inplace=True)
train_data['emp_length'].replace('8 years', 8.0, inplace=True)
train_data['emp_length'].replace('9 years', 9.0, inplace=True)
train_data['emp_length'].replace('10 years', 10.0, inplace=True)
train_data['emp_length'].replace('10+ years', 15.0, inplace=True)
train_data['emp_length'] = train_data['emp_length'].fillna(0)
train_data['pub_rec_bankruptcies'] = train_data['pub_rec_bankruptcies'].fillna(0)
train_data['verification_status'].replace('Verified', 1, inplace=True)
train_data['verification_status'].replace('Source Verified', 1, inplace=True)
train_data['verification_status'].replace('Not Verified', 0, inplace=True)
train_data['initial_list_status'].replace('w', 1, inplace=True)
train_data['initial_list_status'].replace('f', 0, inplace=True)

test_data['sub_grade'] = labelencoder2.fit_transform(test_data['sub_grade'])
test_data['emp_length'].replace('< 1 year', 0, inplace=True)
test_data['emp_length'].replace('1 year', 1.0, inplace=True)
test_data['emp_length'].replace('2 years', 2.0, inplace=True)
test_data['emp_length'].replace('3 years', 3.0, inplace=True)
test_data['emp_length'].replace('4 years', 4.0, inplace=True)
test_data['emp_length'].replace('5 years', 5.0, inplace=True)
test_data['emp_length'].replace('6 years', 6.0, inplace=True)
test_data['emp_length'].replace('7 years', 7.0, inplace=True)
test_data['emp_length'].replace('8 years', 8.0, inplace=True)
test_data['emp_length'].replace('9 years', 9.0, inplace=True)
test_data['emp_length'].replace('10 years', 10.0, inplace=True)
test_data['emp_length'].replace('10+ years', 15.0, inplace=True)
test_data['emp_length'] = test_data['emp_length'].fillna(0)
test_data['pub_rec_bankruptcies'] = test_data['pub_rec_bankruptcies'].fillna(0)
test_data['verification_status'].replace('Verified', 1, inplace=True)
# technically encoding two categories into one... might want to change
test_data['verification_status'].replace('Verified', 1, inplace=True)
test_data['verification_status'].replace('Source Verified', 1, inplace=True)
test_data['verification_status'].replace('Not Verified', 0, inplace=True)
test_data['initial_list_status'].replace('w', 1, inplace=True)
test_data['initial_list_status'].replace('f', 0, inplace=True)

train_data['mort_acc'] = train_data['mort_acc'].fillna(0)

test_data['mort_acc'] = test_data['mort_acc'].fillna(0)
# Strip percent(%) from int_rate
train_data['int_rate'] = train_data['int_rate'].str.rstrip('%').astype(float)
test_data['int_rate'] = test_data['int_rate'].str.rstrip('%').astype(float)

#Strip percent(%) from revol_util
train_data['revol_util'] = train_data['revol_util'].str.rstrip('%').astype(float)
test_data['revol_util'] = test_data['revol_util'].str.rstrip('%').astype(float)
train_data['revol_util'] = train_data['revol_util'].fillna(0)
test_data['revol_util'] = test_data['revol_util'].fillna(0)

X_train = train_data.iloc[:,:-1]
y_train = train_data.iloc[:,-1]
X_test = test_data.iloc[:,:]
X_train_numeric = X_train.select_dtypes(include=np.number)
X_test_numeric = X_test.select_dtypes(include=np.number)
# y_train_numeric = y_train.select_dtypes(include=np.number)
y_train_numeric = y_train.copy(deep=False)
y_train_numeric.replace('Fully Paid', 0.0, inplace=True)
y_train_numeric.replace('Charged Off', 1.0, inplace=True)


BEFORE
(197250, 23)
loan_amnt                      5000
term_(months)                    36
int_rate                     10.65%
installment                  162.87
sub_grade                        B2
emp_length                10+ years
home_ownership                 RENT
annual_inc                  24000.0
verification_status        Verified
purpose                 credit_card
zip_code                      860xx
addr_state                       AZ
dti                           27.65
open_acc                          3
pub_rec                           0
revol_bal                     13648
revol_util                    83.7%
total_acc                         9
initial_list_status               f
application_type         Individual
mort_acc                        NaN
pub_rec_bankruptcies            0.0
loan_status              Fully Paid
Name: 0, dtype: object
AFTER
(197250, 76)
Index(['loan_amnt', 'term_(months)', 'int_rate', 'installment', 'sub_grade',
       'emp_length', 'home_owners

CNNs take vectors with values in [0,1]. Here we normalize the numberical data accordingly with min-max normalization.
Standardize an arbitrary feature X by using

X' = (X - X_min)/(X_max - X_min)

In [3]:
for col in X_train_numeric.columns:
    if X_train_numeric[col].isnull().values.any() > 0:
        print(col)
    x_min = X_train_numeric[col].min()
    x_max = X_train_numeric[col].max()
    X_train_numeric[col] = (X_train_numeric[col] - x_min) / (x_max - x_min)
    # also need to normalize test data, but must use training data
    X_test_numeric[col] = (X_test_numeric[col] - x_min) / (x_max - x_min)
    X_test_numeric[col].mask(X_test_numeric[col] < 0, 0, inplace=True)
    X_test_numeric[col].mask(X_test_numeric[col] > 1, 1, inplace=True)
    # TODO: DETERMINE WHAT TO DO IN THESE CASES!!!
    if X_test_numeric[col].min() < 0:
        print(f"min is less than zero in test data in column {col}")
        print(X_test_numeric[col].min())
    if X_test_numeric[col].max() > 1:
        print(f"max is more than one in test data in column {col}")
        print(X_test_numeric[col].max())

In [4]:
import torch.nn as nn


model = nn.Sequential(
    nn.Conv2d(in_channels=1, out_channels=64, kernel_size=(1, 3), stride=1, padding=0),
    nn.BatchNorm2d(64),
    nn.ReLU(inplace=True),
    nn.MaxPool2d(kernel_size=(1, 2), stride=1),
    nn.Conv2d(in_channels=64, out_channels=64, kernel_size=(1, 3), stride=1, padding=0),
    nn.BatchNorm2d(64),
    nn.ReLU(inplace=True),
    nn.MaxPool2d(kernel_size=(1, 2), stride=1),
    nn.Flatten(),
    nn.Linear(4224, 512),
    nn.ReLU(),
    nn.Dropout(p=0.25),
    nn.Linear(512, 1),
    nn.Sigmoid()
)

In [5]:
# our model has some # of parameters:
count = 0
for p in model.parameters():
    n_params = np.prod(list(p.data.shape)).item()
    count += n_params
print(f'total params: {count}')

total params: 2176577


In [6]:
import torch.optim as optim
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters())

In [7]:
print(X_train_numeric.values)

[[0.13043478 0.         0.25339147 ... 0.         0.         0.        ]
 [0.05797101 1.         0.47722868 ... 0.         0.         0.        ]
 [0.05507246 0.         0.51065891 ... 0.         0.         0.        ]
 ...
 [0.30434783 0.         0.42005814 ... 0.         0.         0.        ]
 [0.89565217 1.         0.84932171 ... 0.         0.12903226 0.        ]
 [0.14492754 0.         0.42005814 ... 0.         0.         0.        ]]


Need a Dataset object, which we make from the dataframes

In [8]:
# Custom Dataset
from torch.utils.data import Dataset
import torch


class CustomDataset(Dataset):
    def __init__(self, X, Y):
        """
        X: a dataframe with rows as training examples and columns features
        Y: a pandas series with labels"""
        self.x_train=torch.tensor(X.values, dtype=torch.float32)
        self.y_train=torch.tensor(Y.values, dtype=torch.float32)
        self.x_train = torch.reshape(self.x_train, (self.x_train.shape[0], 1, 1, self.x_train.shape[1],))

    def __len__(self):
        return len(self.y_train)

    def __getitem__(self,idx):
        return (self.x_train[idx], self.y_train[idx])

Make the dataloaders

In [9]:
from torch.utils.data import DataLoader, random_split

BATCH_SIZE = 512

ds_train = CustomDataset(X_train_numeric, y_train_numeric)
len_full = len(ds_train)
len_train = int(0.8*len(ds_train))
len_val = len_full - len_train
ds_train, ds_val = random_split(ds_train, [len_train, len_val])
train_loader = DataLoader(ds_train, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(ds_val, batch_size=BATCH_SIZE, shuffle=True)
val_eval_loader = DataLoader(ds_val, batch_size=len_val, shuffle=False)

In [10]:
n_epochs = 5  # paper used 100 originally
lr = 0.001

train_loss_hist = np.zeros([n_epochs, 1])
val_loss_hist = np.zeros([n_epochs, 1])
for epoch in range(n_epochs):
    print(f'EPOCH {epoch}')
    model.train()
    train_loss = 0
    for i, data in enumerate(train_loader):
        features, labels = data
        # print(type(features))
        # print(f"PESKY BASTARDS: {features.isnan().sum()}")
        optimizer.zero_grad()
        # forward pass
        output = model(features)
        output = torch.reshape(output, (output.shape[0], ))
        if output.isnan().sum() > 0:
            print(f"RIP{output.isnan().sum()}")
        output = torch.nan_to_num(output)
        # calculate loss
        loss = criterion(output, labels)
        # backward pass
        loss.backward()
        # update
        optimizer.step()
        # track training loss
        train_loss_hist[epoch] += loss.item()
    train_loss_hist[epoch] /= len(train_loader)
    print(f"\ttraining loss: {train_loss_hist[epoch]}")

    # validate
    with torch.no_grad():
        model.eval()
        output = None
        labels = None
        for i, data in enumerate(val_eval_loader):
            features, labels = data
            output = model(features)
            output = torch.reshape(output, (output.shape[0], ))
            output = torch.nan_to_num(output)
            loss = criterion(output, labels)
            val_loss_hist[epoch] += loss.item()
        preds = output.cpu().detach().numpy()
        ground_truths = labels.cpu().detach().numpy()
        print(f'\tAUC: {sklearn.metrics.roc_auc_score(ground_truths, preds)}')
        val_loss_hist[epoch] /= len(val_eval_loader)
        print(f"\tvalidation loss: {val_loss_hist[epoch]}")

EPOCH 0


KeyboardInterrupt: 

In [ ]:
import torch

with torch.no_grad():
    model.eval()
    for i, data in enumerate(val_eval_loader):
        print(i)
        features, labels = data
        output = model(features)
        output = torch.reshape(output, (output.shape[0], ))
        # output = torch.nan_to_num(output)
        preds = output.cpu().detach().numpy()
        ground_truths = labels.cpu().detach().numpy()
        print(f'AUC: {sklearn.metrics.roc_auc_score(ground_truths, preds)}')
        

0
AUC: 0.6812805903216952
